In [1]:
import numpy as np
import potentials
import atomman as am
import iprPy

## Load the database

The database to use where the calculation records will be added to and to search for existing calculations to skip.

In [4]:
database = iprPy.load_database('master')
print(database)

database style mongo at localhost:27017.iprPy


## Required terms

These are the basic terms required by most calculations in the workflow

- __lammps_command__ is the primary LAMMPS executable to use.
- __mpi_command__ is the MPI command to use.  Leave {np_per_runner} as a variable.

In [3]:
prepare_terms = {}
prepare_terms['lammps_command'] = 'lmp_mpi'
prepare_terms['mpi_command'] = 'C:/Program Files/MPICH2/bin/mpiexec -localonly {np_per_runner}'

## Optional: Alternate LAMMPS executables

These are alternate LAMMPS executables to use with interatomic potentials that are not compatible with the primary executable set above.  
- __lammps_command_snap_1__ asdf
- __lammps_command_snap_1__ asdf
- __lammps_command_old__ asdf

In [4]:
#prepare_terms['lammps_command_snap_1'] = 'C:/Program Files/LAMMPS/2017-01-27/bin/lmp_mpi'
#prepare_terms['lammps_command_snap_2'] = 'C:/Program Files/LAMMPS/2019-06-05/bin/lmp_mpi'
#prepare_terms['lammps_command_old'] = 'C:/Program Files/LAMMPS/2019-06-05/bin/lmp_mpi'

## Optional: Potential limiters

Any term keys that start with "potential_" are passed into get_lammps_potential() as kwargs (without the potential_ prefix) allowing for parsing of the potentials which will be used for preparing the calculations.

Examples used here:
- __potential_status__ limits the included potential version status.  Values are all, active, superseded or retracted. Default value is active, which is best for most users as it only includes the newest versions of the available potentials.
- __potential_id__ limits the potentials used to only the ones explicitly listed by id.
- __potential_pair_style__ limits the potentials used to only the ones with a LAMMPS pair_style included in the list.

In [5]:
prepare_terms['potential_status'] = 'all'

#prepare_terms['potential_id'] = [
#    '1999--Mishin-Y--Ni--LAMMPS--ipr1',
#    '1999--Mishin-Y--Al--LAMMPS--ipr1',
    #'EAM_CubicNaturalSpline_ErcolessiAdams_1994_Al__MO_800509458712_002',
#]

#prepare_terms['potential_pair_style'] = [
#    'eam', 'eam/alloy', 'eam/fs', 'eam/cd',
#]     

In [8]:
lmppots, lmppots_df = database.potdb.get_lammps_potentials(return_df=True, status=None)
all_lmppot_ids = np.unique(lmppots_df.id).tolist()
print(len(all_lmppot_ids), 'potential ids found')


860 potential ids found


In [7]:
# Define function for iterating over subsets of potentials
def yield_lmppot_ids(delta=20):
    for i in range(delta, len(all_lmppot_ids), delta):
        print(f'Using potential #s {i-delta} to {i-1}\n')
        yield all_lmppot_ids[i-delta:i]
    print(f'Using potential #s {i} to {len(all_lmppot_ids)-1}\n')
    yield all_lmppot_ids[i:len(all_lmppot_ids)]


## Calculation Pools

The prepared calculations are divided into separate "pools" where the prepared calculations will be executed from.  

By default, the iprPy runner methods are all assigned a set number of processors to work with, and then runs through the calculations in a pool by randomly selecting them. As such, individual pools should be used for different steps along the workflow, as well as for calculations that will be assigned different numbers of processors.

Each pool is defined with the following master prepare parameters
- __styles__ lists the iprPy calculation styles to prepare in the pool.  By default, these will use the pre-defined "main" branch, but alternate branches can be selected by giving the branch name after a :.
- __run_directory__ is the name of the specific run directory where the pool is located.  All prepared calculations will be created in this run directory.
- __np_per_runner__ is the number of processors each runner will be assigned to use for the underlying simulations.

__NOTE:__ These values can either be singular or lists of equal length.  Giving list values allows for multiple pools to be prepared with the same master_prepare call.  This Notebook updates the values and only builds one pool at a time for more interactivity.

### Pool #1: Basic potential evaluations and scans

Styles

- __isolated_atom__ evaluates the energy of a single atom in isolation.
- __diatom_scan__ evaluates the energy of a pair of atoms at various interatomic spacings.
- __E_vs_r_scan__ evaluates the energy of crystal prototypes subjected to a volumetric scan.
- __E_vs_r_scan:bop__ evaluates the energy of crystal prototypes subjected to a volumetric scan. Uses a higher rmin to avoid extremely costly energy evaluations for bop potentials.

In [6]:
styles = [
    'isolated_atom',
    'diatom_scan',
    'E_vs_r_scan',
    'E_vs_r_scan:bop',
]
prepare_terms['styles']        = ' '.join(styles)
prepare_terms['run_directory'] = 'master_1'
prepare_terms['np_per_runner'] = '1'
database.master_prepare(**prepare_terms)

Preparing calculation isolated_atom branch main
0 existing calculation records found
2 matching interatomic potentials found
2 calculation combinations to check
2 new records to prepare

Preparing calculation diatom_scan branch main
0 existing calculation records found
2 matching interatomic potentials found
2 calculation combinations to check
2 new records to prepare

Preparing calculation E_vs_r_scan branch main
10 existing calculation records found
19 matching crystal prototypes found
2 matching interatomic potentials found
20 calculation combinations to check
10 new records to prepare

Preparing calculation E_vs_r_scan branch bop
20 existing calculation records found
19 matching crystal prototypes found
0 matching interatomic potentials found
1 invalid calculations skipped
0 calculation combinations to check



### Pool #2: Round 1 of crystal structure relaxations

Styles

- __relax_box__ relaxes a structure by only adjusting the box dimensions and not the atom positions.
- __relax_static__ relaxes a structure using energy/force minimizations. 
- __relax_dynamic__ relaxes a structure using dynamic Langevin energy damping, with only a single "kick" at the beginning.

In [7]:
styles = [
    'relax_box',
    'relax_static',
    'relax_dynamic',
]
prepare_terms['styles']        = ' '.join(styles)
prepare_terms['run_directory'] = 'master_2'
prepare_terms['np_per_runner'] = '1'
database.master_prepare(**prepare_terms)

Preparing calculation relax_box branch main
39 existing calculation records found
6587 matching atomic references found
2 matching interatomic potentials found
2 matching interatomic potentials found
10 matching atomic parents found
66 calculation combinations to check
27 new records to prepare

Preparing calculation relax_static branch main
78 existing calculation records found
6587 matching atomic references found
2 matching interatomic potentials found
2 matching interatomic potentials found
10 matching atomic parents found
66 calculation combinations to check
27 new records to prepare

Preparing calculation relax_dynamic branch main
39 existing calculation records found
6587 matching atomic references found
2 matching interatomic potentials found
2 matching interatomic potentials found
10 matching atomic parents found
66 calculation combinations to check
27 new records to prepare



### Pool #3: Round 2 of crystal structure relaxations

Styles

- __relax_static:from_dymanic__ further relaxes the results of the relax_dynamic calculation above using the relax_static method. 

In [8]:
styles = [
    'relax_static:from_dynamic'
]
prepare_terms['styles']        = ' '.join(styles)
prepare_terms['run_directory'] = 'master_3'
prepare_terms['np_per_runner'] = '1'
database.master_prepare(**prepare_terms)

Preparing calculation relax_static branch from_dynamic
105 existing calculation records found
2 matching interatomic potentials found
66 matching atomic archives found
66 calculation combinations to check
27 new records to prepare



### Pool #4: Crystal space group analysis

Styles

- __crystal_space_group:prototype__ performs a space group analysis on the prototype reference structures in the database.
- __crystal_space_group:reference__ performs a space group analysis on any DFT reference structures in the database.
- __crystal_space_group:relax__ performs a space group analysis on all of the results of the relax_box, relax_static and relax_static:from dynamic calculations.

In [9]:
num_lmppot_ids = 50
styles = [
    #'crystal_space_group:prototype',
    #'crystal_space_group:reference',
    'crystal_space_group:relax',
]
prepare_terms['styles']        = ' '.join(styles)
prepare_terms['run_directory'] = 'master_4'
prepare_terms['np_per_runner'] = '1'
for lmppot_ids in yield_lmppot_ids(num_lmppot_ids):
    prepare_terms['potential_id'] = lmppot_ids
    database.master_prepare(**prepare_terms)

Using potential #s 0 to 49

Preparing calculation crystal_space_group branch relax
112360 existing calculation records found
8507 matching atomic archives found
5956 matching atomic archives found
14463 calculation combinations to check
5956 new records to prepare

Using potential #s 50 to 99

Preparing calculation crystal_space_group branch relax
118316 existing calculation records found
4278 matching atomic archives found
3170 matching atomic archives found
7448 calculation combinations to check
3261 new records to prepare

Using potential #s 100 to 149

Preparing calculation crystal_space_group branch relax
121577 existing calculation records found
5343 matching atomic archives found
3816 matching atomic archives found
9159 calculation combinations to check
3816 new records to prepare

Using potential #s 150 to 199

Preparing calculation crystal_space_group branch relax
125393 existing calculation records found
6139 matching atomic archives found
5276 matching atomic archives found


- - -

### NOTE: 

All pools after this point use relaxed_crystal structures as the structure inputs.  These are generated from the relaxed structures and the associated crystal space group calculations using the !(process-relaxed method)!

### Pool #5: Elastic constants and phonons

Styles

- __elastic_constants_static__ computes the static elastic constants of a crystal structure using small strains.
- __phonon__ uses phonopy to evaluate the phonons of a crystal structure and performs a quasiharmonic analysis if possible.

In [10]:
styles = [
    'elastic_constants_static',
    'phonon',
]
prepare_terms['styles']        = ' '.join(styles)
prepare_terms['run_directory'] = 'master_5'
prepare_terms['np_per_runner'] = '1'
database.master_prepare(**prepare_terms)

Preparing calculation elastic_constants_static branch main
30 existing calculation records found
2 matching interatomic potentials found
17 matching atomic parents found
51 calculation combinations to check
21 new records to prepare

Preparing calculation phonon branch main
0 existing calculation records found
2 matching interatomic potentials found
17 matching atomic parents found
17 calculation combinations to check
17 new records to prepare



### Pool #6: Simple crystal defects

__NOTE__: While these styles can be prepared similarly to the others, it is much faster to pre-select the parent relaxed_crystal records based on their associated prototypes.  Therefore, this code calls master_prepare separately for each style+family combination.


Styles
- __point_defect_static__ computes point defect formation energies and elastic dipole tensors.
- __surface_energy_static__ computes free surface formation energies.
- __stacking_fault_map_2D__ computes 2D stacking fault energy maps a.k.a. gamma surfaces.


In [5]:
styles = [
    'point_defect_static',
    'surface_energy_static',
    #'stacking_fault_map_2D',
]

prepare_terms['run_directory'] = 'master_6'
prepare_terms['np_per_runner'] = '1'

for style in styles:
    print(style)
    prepare_terms['styles'] = style
    
    # Get defect reference records
    if style == 'point_defect_static':
        defects = database.get_records_df(style='point_defect')
    elif style == 'surface_energy_static':
        defects = database.get_records_df(style='free_surface')
    elif style == 'stacking_fault_map_2D':
        defects = database.get_records_df(style='stacking_fault')
    
    # Loop over reference record family values
    for family in np.unique(defects.family):
        print(family)
        prepare_terms['parent_family'] = family
        prepare_terms['defect_family'] = family
        database.master_prepare(**prepare_terms)

del prepare_terms['parent_family']
del prepare_terms['defect_family']

point_defect_static
A1--Cu--fcc
Preparing calculation point_defect_static branch main
0 existing calculation records found
2 matching interatomic potentials found
2 matching atomic parents found
9 matching defects
18 calculation combinations to check
18 new records to prepare

A2--W--bcc
Preparing calculation point_defect_static branch main
18 existing calculation records found
2 matching interatomic potentials found
0 matching atomic parents found
9 matching defects
9 invalid calculations skipped
0 calculation combinations to check

A3--Mg--hcp
Preparing calculation point_defect_static branch main
18 existing calculation records found
2 matching interatomic potentials found
2 matching atomic parents found
12 matching defects
24 calculation combinations to check
20 new records to prepare

A4--C--dc
Preparing calculation point_defect_static branch main
38 existing calculation records found
2 matching interatomic potentials found
0 matching atomic parents found
8 matching defects
8 inval

### Pool #7: Dislocation core constructions

Styles

- __dislocation_monopole:fcc_edge_100__ builds a dislocation monopole system for an fcc edge dislocation on a (100) plane.
- __dislocation_monopole:bcc_screw__ builds a dislocation monopole system for a bcc screw dislocation on a (110) plane.
- __dislocation_monopole:bcc_edge__ builds a dislocation monopole system for a bcc edge dislocation on a (110) plane.
- __dislocation_monopole:bcc_edge_112__ builds a dislocation monopole system for a bcc edge dislocation on a (112) plane.
- __dislocation_periodic_array:fcc_edge_mix__ builds a periodic array of dislocations system for an fcc edge or mixed dislocation on a (111) plane.
- __dislocation_periodic_array:fcc_screw__ builds a periodic array of dislocations system for an fcc screw dislocation on a (111) plane.

In [6]:
styles = [
    'dislocation_monopole:fcc_edge_100',
    'dislocation_monopole:bcc_screw',
    'dislocation_monopole:bcc_edge',
    'dislocation_monopole:bcc_edge_112',
    'dislocation_periodic_array:fcc_edge_mix',
    'dislocation_periodic_array:fcc_screw',
]

prepare_terms['styles']        = ' '.join(styles)
prepare_terms['run_directory'] = 'master_7'
prepare_terms['np_per_runner'] = '8'
prepare_terms['annealsteps'] = '100000'
database.master_prepare(**prepare_terms)

Preparing calculation dislocation_monopole branch fcc_edge_100
1 existing calculation records found
1 matching interatomic potentials found
1 matching atomic archives found
1 matching defects
1 calculation combinations to check
1 new records to prepare

Preparing calculation dislocation_monopole branch bcc_screw
2 existing calculation records found
1 matching interatomic potentials found
0 matching atomic archives found
1 matching defects
1 invalid calculations skipped
0 calculation combinations to check

Preparing calculation dislocation_monopole branch bcc_edge
2 existing calculation records found
1 matching interatomic potentials found
0 matching atomic archives found
1 matching defects
1 invalid calculations skipped
0 calculation combinations to check

Preparing calculation dislocation_monopole branch bcc_edge_112
2 existing calculation records found
1 matching interatomic potentials found
0 matching atomic archives found
1 matching defects
1 invalid calculations skipped
0 calculat

### Pool #8: Temperature-dependent crystal relaxations

Styles

- __relax_dynamic:at_temp__ 


---

## Database management methods

### Check record statuses

In [16]:
for style in [
    #'diatom_scan',
    #'isolated_atom',
    #'E_vs_r_scan',
    #'relax_box',
    #'relax_static',
    #'relax_dynamic',
    #'crystal_space_group',
    #'elastic_constants_static',
    #'phonon',
    'point_defect_static',
    #'surface_energy_static',
    #'stacking_fault_map_2D',
    #'dislocation_monopole',
    #'dislocation_periodic_array',
    
]:
    database.check_records(f'calculation_{style}')

In database style local at E:\calculations\ipr\test:
- 38 of style calculation_point_defect_static
 - 1 finished
 - 36 not finished
 - 1 issued errors


### Delete all records

In [7]:
for style in [
    'diatom_scan',
    'isolated_atom',
    #'E_vs_r_scan',
    #'relax_box',
    #'relax_static',
    #'relax_dynamic',
    #'crystal_space_group',
    #'elastic_constants_static',
    #'phonon',
    #'point_defect_static',
    #'surface_energy_static',
    #'stacking_fault_map_2D',
    #'dislocation_monopole',
    #'dislocation_periodic_array',
    
]:
    database.destroy_records(f'calculation_{style}')

0 records found to be destroyed
0 records found to be destroyed


In [15]:
for style in [
   # 'diatom_scan',
   # 'isolated_atom',
   # 'E_vs_r_scan',
   # 'relax_box',
   # 'relax_static',
   # 'relax_dynamic',
   # 'crystal_space_group',
   # 'elastic_constants_static',
   # 'phonon',
    'point_defect_static',
   # 'surface_energy_static',
   # 'stacking_fault_map_2D',
   # 'dislocation_monopole',
    #'dislocation_periodic_array',
    
]:
    database.clean_records(record_style=f'calculation_{style}', run_directory='master_6')

2 records to clean
